# Робота з базою даних SQLite в тестуванні Python

## 1. Вступ до SQLite

**SQLite** — це легка, вбудована реляційна база даних, яка не потребує окремого серверного процесу. Ідеально підходить для тестування та розробки.

### Переваги SQLite для тестування:
- Не потребує встановлення додаткового ПЗ
- Зберігається в одному файлі
- Швидка робота з невеликими обсягами даних
- Підтримує стандарт SQL
- Легко створювати тимчасові бази для тестів

## 2. Підключення до SQLite в Python

### Базове підключення

```python
import sqlite3

# Створення/підключення до бази даних
conn = sqlite3.connect('test_database.db')

# Створення курсора для виконання SQL-запитів
cursor = conn.cursor()

# Виконання запиту
cursor.execute("SELECT sqlite_version()")
version = cursor.fetchone()
print(f"SQLite версія: {version[0]}")

# Закриття з'єднання
conn.close()
```

### Використання контекстного менеджера

```python
import sqlite3

with sqlite3.connect('test_database.db') as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM users")
    # З'єднання автоматично закриється
```

## 3. Створення та наповнення тестової бази даних

### Створення таблиці

```python
import sqlite3

conn = sqlite3.connect('test_users.db')
cursor = conn.cursor()

# Створення таблиці користувачів
cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT NOT NULL UNIQUE,
        email TEXT NOT NULL,
        age INTEGER,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
''')

conn.commit()
conn.close()
```

### Вставка даних

```python
# Вставка одного запису
cursor.execute('''
    INSERT INTO users (username, email, age) 
    VALUES (?, ?, ?)
''', ('john_doe', 'john@example.com', 25))

# Вставка кількох записів
users_data = [
    ('alice', 'alice@example.com', 30),
    ('bob', 'bob@example.com', 28),
    ('charlie', 'charlie@example.com', 35)
]

cursor.executemany('''
    INSERT INTO users (username, email, age) 
    VALUES (?, ?, ?)
''', users_data)

conn.commit()
```

## 4. Тестування операцій з базою даних

### Приклад класу для роботи з користувачами

```python
# user_manager.py
import sqlite3

class UserManager:
    def __init__(self, db_name='users.db'):
        self.db_name = db_name
        self._init_db()
    
    def _init_db(self):
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS users (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    username TEXT NOT NULL UNIQUE,
                    email TEXT NOT NULL,
                    age INTEGER
                )
            ''')
            conn.commit()
    
    def add_user(self, username, email, age):
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            cursor.execute(
                'INSERT INTO users (username, email, age) VALUES (?, ?, ?)',
                (username, email, age)
            )
            conn.commit()
            return cursor.lastrowid
    
    def get_user(self, user_id):
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            cursor.execute('SELECT * FROM users WHERE id = ?', (user_id,))
            return cursor.fetchone()
    
    def get_all_users(self):
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            cursor.execute('SELECT * FROM users')
            return cursor.fetchall()
    
    def delete_user(self, user_id):
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            cursor.execute('DELETE FROM users WHERE id = ?', (user_id,))
            conn.commit()
            return cursor.rowcount > 0
```

### Написання тестів з unittest

```python
# test_user_manager.py
import unittest
import os
import sqlite3
from user_manager import UserManager

class TestUserManager(unittest.TestCase):
    
    def setUp(self):
        """Виконується перед кожним тестом"""
        self.test_db = 'test_users.db'
        self.manager = UserManager(self.test_db)
    
    def tearDown(self):
        """Виконується після кожного тесту"""
        if os.path.exists(self.test_db):
            os.remove(self.test_db)
    
    def test_add_user(self):
        """Тест додавання користувача"""
        user_id = self.manager.add_user('testuser', 'test@example.com', 25)
        self.assertIsNotNone(user_id)
        self.assertGreater(user_id, 0)
    
    def test_get_user(self):
        """Тест отримання користувача"""
        user_id = self.manager.add_user('alice', 'alice@example.com', 30)
        user = self.manager.get_user(user_id)
        
        self.assertIsNotNone(user)
        self.assertEqual(user[1], 'alice')  # username
        self.assertEqual(user[2], 'alice@example.com')  # email
        self.assertEqual(user[3], 30)  # age
    
    def test_get_all_users(self):
        """Тест отримання всіх користувачів"""
        self.manager.add_user('user1', 'user1@example.com', 20)
        self.manager.add_user('user2', 'user2@example.com', 25)
        
        users = self.manager.get_all_users()
        self.assertEqual(len(users), 2)
    
    def test_delete_user(self):
        """Тест видалення користувача"""
        user_id = self.manager.add_user('to_delete', 'delete@example.com', 22)
        result = self.manager.delete_user(user_id)
        
        self.assertTrue(result)
        self.assertIsNone(self.manager.get_user(user_id))
    
    def test_duplicate_username(self):
        """Тест на унікальність username"""
        self.manager.add_user('unique', 'unique@example.com', 25)
        
        with self.assertRaises(sqlite3.IntegrityError):
            self.manager.add_user('unique', 'another@example.com', 30)

if __name__ == '__main__':
    unittest.main()
```

### Написання тестів з pytest

```python
# test_user_manager_pytest.py
import pytest
import os
import sqlite3
from user_manager import UserManager

@pytest.fixture
def user_manager():
    """Фікстура для створення та очищення тестової БД"""
    test_db = 'test_users_pytest.db'
    manager = UserManager(test_db)
    
    yield manager
    
    # Очищення після тесту
    if os.path.exists(test_db):
        os.remove(test_db)

def test_add_user(user_manager):
    """Тест додавання користувача"""
    user_id = user_manager.add_user('testuser', 'test@example.com', 25)
    assert user_id is not None
    assert user_id > 0

def test_get_user(user_manager):
    """Тест отримання користувача"""
    user_id = user_manager.add_user('alice', 'alice@example.com', 30)
    user = user_manager.get_user(user_id)
    
    assert user is not None
    assert user[1] == 'alice'
    assert user[2] == 'alice@example.com'
    assert user[3] == 30

def test_delete_user(user_manager):
    """Тест видалення користувача"""
    user_id = user_manager.add_user('to_delete', 'delete@example.com', 22)
    result = user_manager.delete_user(user_id)
    
    assert result is True
    assert user_manager.get_user(user_id) is None

def test_duplicate_username(user_manager):
    """Тест на унікальність username"""
    user_manager.add_user('unique', 'unique@example.com', 25)
    
    with pytest.raises(sqlite3.IntegrityError):
        user_manager.add_user('unique', 'another@example.com', 30)
```

## 5. Використання in-memory бази даних

Для швидких тестів можна використовувати базу даних у пам'яті:

```python
import sqlite3

# База даних у пам'яті (не зберігається на диску)
conn = sqlite3.connect(':memory:')

# Або
conn = sqlite3.connect('file::memory:?cache=shared', uri=True)
```

### Приклад тесту з in-memory БД

```python
import unittest
import sqlite3

class TestInMemoryDB(unittest.TestCase):
    
    def setUp(self):
        self.conn = sqlite3.connect(':memory:')
        self.cursor = self.conn.cursor()
        
        # Створення таблиці
        self.cursor.execute('''
            CREATE TABLE products (
                id INTEGER PRIMARY KEY,
                name TEXT NOT NULL,
                price REAL
            )
        ''')
        self.conn.commit()
    
    def tearDown(self):
        self.conn.close()
    
    def test_insert_product(self):
        self.cursor.execute(
            'INSERT INTO products (name, price) VALUES (?, ?)',
            ('Laptop', 999.99)
        )
        self.conn.commit()
        
        self.cursor.execute('SELECT * FROM products WHERE name = ?', ('Laptop',))
        product = self.cursor.fetchone()
        
        self.assertIsNotNone(product)
        self.assertEqual(product[1], 'Laptop')
        self.assertEqual(product[2], 999.99)
```

## 6. Мокування та тестування з unittest.mock

```python
from unittest.mock import Mock, patch
import sqlite3

def test_database_error_handling():
    """Тест обробки помилок бази даних"""
    with patch('sqlite3.connect') as mock_connect:
        # Симулюємо помилку підключення
        mock_connect.side_effect = sqlite3.Error("Connection failed")
        
        with pytest.raises(sqlite3.Error):
            conn = sqlite3.connect('test.db')
```

## 7. Найкращі практики тестування БД

### 1. Ізоляція тестів
- Кожен тест повинен мати свою чисту базу даних
- Використовуйте `setUp()` та `tearDown()` для ініціалізації та очищення

### 2. Використання транзакцій

```python
def test_with_rollback():
    conn = sqlite3.connect('test.db')
    try:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO users VALUES (?, ?, ?)', (1, 'test', 'test@example.com'))
        # Якщо тест провалився, транзакція не буде застосована
        conn.rollback()
    finally:
        conn.close()
```

### 3. Перевірка даних після операцій

```python
def test_user_update():
    manager = UserManager()
    user_id = manager.add_user('john', 'john@example.com', 25)
    
    # Оновлення
    manager.update_user(user_id, age=26)
    
    # Перевірка
    user = manager.get_user(user_id)
    assert user[3] == 26
```

### 4. Тестування крайових випадків

```python
def test_empty_database(user_manager):
    """Тест з порожньою БД"""
    users = user_manager.get_all_users()
    assert len(users) == 0

def test_nonexistent_user(user_manager):
    """Тест отримання неіснуючого користувача"""
    user = user_manager.get_user(999)
    assert user is None
```

## 8. Корисні SQL-запити для тестування

```python
# Перевірка кількості записів
cursor.execute('SELECT COUNT(*) FROM users')
count = cursor.fetchone()[0]

# Очищення таблиці
cursor.execute('DELETE FROM users')
cursor.execute('DELETE FROM sqlite_sequence WHERE name="users"')  # Скидання автоінкременту

# Перевірка структури таблиці
cursor.execute('PRAGMA table_info(users)')
columns = cursor.fetchall()

# Отримання всіх таблиць у БД
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()
```

## 9. Завдання для практики

1. Створіть клас `ProductManager` для роботи з таблицею товарів
2. Напишіть тести для всіх CRUD операцій
3. Додайте метод пошуку товарів за ціною
4. Створіть тести для перевірки обмежень (constraints)
5. Реалізуйте тести з використанням in-memory бази даних

## 10. Додаткові ресурси

- Документація SQLite: https://www.sqlite.org/docs.html
- Python sqlite3: https://docs.python.org/3/library/sqlite3.html
- Pytest документація: https://docs.pytest.org/
- Unittest документація: https://docs.python.org/3/library/unittest.html

---

## Висновок

Робота з SQLite в тестуванні Python дозволяє:
- Швидко створювати тестові бази даних
- Ізолювати тести один від одного
- Перевіряти коректність роботи з даними
- Не залежати від зовнішніх сервісів БД

**Пам'ятайте**: завжди очищайте тестові дані після виконання тестів!